<a href="https://colab.research.google.com/github/campospluiza/procesamientoDelHabla/blob/main/Campos_Desafio3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.



In [1]:
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# Cargar los datos (sin encabezados, pies de página o citas)
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [3]:
# Instanciar el vectorizador
tfidfvect = TfidfVectorizer()

In [4]:
# Vectorizar los documentos
X_train = tfidfvect.fit_transform(newsgroups.data)

In [5]:
# Tomar 5 documentos al azar
random_indices = random.sample(range(len(newsgroups.data)), 5)
selected_docs = [newsgroups.data[i] for i in random_indices]
selected_labels = [newsgroups.target[i] for i in random_indices]

print("Documentos seleccionados y sus etiquetas:")
for idx in random_indices:
    print(f"\nÍndice: {idx}")
    print(f"Texto: {newsgroups.data[idx][:300]}...")  # Muestra solo los primeros 300 caracteres
    print(f"Etiqueta: {newsgroups.target[idx]} - {newsgroups.target_names[newsgroups.target[idx]]}")

Documentos seleccionados y sus etiquetas:

Índice: 5965
Texto: It is model number #7033D, a 14" interlaced .28dp.  BTW, if you have a
number to contact the company, that would really be helpful to.  Thanks
for replying.  I was beginning to believe that I was never going to get
a reply.  I posted this on the netnews bboard because the first message
I sent to you...
Etiqueta: 3 - comp.sys.ibm.pc.hardware

Índice: 10457
Texto: If you are interested in receiving the final player stats for
the 1992-1993 NHL Season as well as playoff box scores, stats and
scores/updates...
then e-mail my stat server

mmilitzo@skidmore.edu...
Etiqueta: 10 - rec.sport.hockey

Índice: 8926
Texto:  >With that in mind...

I just recently purchased the GCC BLP Elite and I really like it.  My
needs
are much the same as what you describe.  In addition, I wanted to get one
that
I could access via AppleTalk, so that eliminated the new line of
inexpensive 
printers from Apple.  The print quality is ...
Etiqueta: 4 - co

In [6]:
# Medir la similaridad coseno con todos los documentos
similarities = cosine_similarity(X_train[random_indices], X_train)

In [7]:
# Analizar los 5 documentos más similares para cada documento seleccionado
for i, idx in enumerate(random_indices):
    print(f"\nDocumentos más similares para el documento con índice {idx}:")

    # Obtener las similaridades y sus índices
    most_similar_indices = np.argsort(similarities[i])[::-1][1:6]  # Excluyendo el documento en sí

    for sim_idx in most_similar_indices:
        print(f"Índice similar: {sim_idx}")
        print(f"Texto similar: {newsgroups.data[sim_idx][:300]}...")  # Muestra solo los primeros 300 caracteres
        print(f"Etiqueta: {newsgroups.target[sim_idx]} - {newsgroups.target_names[newsgroups.target[sim_idx]]}")


Documentos más similares para el documento con índice 5965:
Índice similar: 9623
Texto similar: Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012
                 Prelude to Current Events in Nagorno-Karabakh

        +---------------------------------------------------------+
        |                                                         |
        |  I saw a naked girl wi...
Etiqueta: 17 - talk.politics.mideast
Índice similar: 6894
Texto similar: Here is a press release from the White House.

 President Clinton's Remarks On Waco With Q/A
 To: National Desk
 Contact: White House Office of the Press Secretary, 202-456-2100

   WASHINGTON, April 20 -- Following are remarks by President 
Clinton in a question and answer session with the press:

...
Etiqueta: 16 - talk.politics.guns
Índice similar: 1332
Texto similar:   That was the entire point to *you*.  What exactly did I claim?
     --------------------------------------------------
    "I've heard eye-witness des

2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [8]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

In [10]:
# Cargar los datos (sin encabezados, pies de página o citas)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [11]:
# Instanciar el vectorizador con parámetros ajustados
tfidfvect = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

# Vectorizar los documentos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
X_test = tfidfvect.transform(newsgroups_test.data)

# Definir etiquetas
y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [12]:
# Definir los modelos a probar
models = {
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB()
}

# Función para entrenar modelos y evaluar el F1-score
def evaluate_models(models, X_train, y_train, X_test, y_test):
    results = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='macro')
        results[name] = f1
        print(f'{name} F1-score (macro): {f1:.4f}')

    return results

# Evaluar los modelos
f1_scores = evaluate_models(models, X_train, y_train, X_test, y_test)

MultinomialNB F1-score (macro): 0.6512
ComplementNB F1-score (macro): 0.6943


In [13]:
# Ajuste de hiperparámetros (Grid Search) para el modelo MultinomialNB
param_grid = {
    'alpha': [0.5, 1.0, 1.5, 2.0]  # Parámetro de suavizado
}

grid_search = GridSearchCV(MultinomialNB(), param_grid, scoring='f1_macro', cv=5)
grid_search.fit(X_train, y_train)

print(f"\nMejores parámetros para MultinomialNB: {grid_search.best_params_}")
print(f"Mejor F1-score (macro) con ajuste: {grid_search.best_score_:.4f}")



Mejores parámetros para MultinomialNB: {'alpha': 0.5}
Mejor F1-score (macro) con ajuste: 0.7199


In [14]:
# Evaluar el modelo con los mejores parámetros
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
f1_best = f1_score(y_test, y_pred_best, average='macro')
print(f"F1-score (macro) del mejor modelo en el conjunto de test: {f1_best:.4f}")

F1-score (macro) del mejor modelo en el conjunto de test: 0.6626


3. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [15]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# Cargar los datos (sin encabezados, pies de página o citas)
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# Instanciar el vectorizador
tfidfvect = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

# Vectorizar los documentos
X_train = tfidfvect.fit_transform(newsgroups.data)

# Transponer la matriz documento-término a matriz término-documento
X_terms = X_train.T

# Obtener el vocabulario (lista de palabras)
idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}

In [17]:
# Elegir 5 palabras de interés manualmente
selected_words = ['computer', 'internet', 'data', 'science', 'research']

# Crear una lista de índices correspondientes a las palabras elegidas
selected_indices = [tfidfvect.vocabulary_.get(word) for word in selected_words]

In [18]:
# Verificar que todas las palabras están en el vocabulario
if None in selected_indices:
    missing_words = [word for word, idx in zip(selected_words, selected_indices) if idx is None]
    print(f"Las siguientes palabras no se encontraron en el vocabulario: {missing_words}")
else:
    # Calcular la similaridad entre palabras
    similarities = cosine_similarity(X_terms[selected_indices])

    # Mostrar las similaridades
    for i, word in enumerate(selected_words):
        print(f"\nPalabra: '{word}' (Índice: {selected_indices[i]})")
        # Obtener los índices de las palabras más similares, excluyendo la misma palabra
        most_similar_indices = np.argsort(similarities[i])[::-1][1:6]  # Excluyendo el propio
        for sim_idx in most_similar_indices:
            similar_word = idx2word[sim_idx]
            print(f"Palabra similar: '{similar_word}' (Índice: {sim_idx}), Similaridad: {similarities[i][sim_idx]:.4f}")


Palabra: 'computer' (Índice: 9789)
Palabra similar: '00000' (Índice: 3), Similaridad: 0.0940
Palabra similar: '000' (Índice: 1), Similaridad: 0.0800
Palabra similar: '0000' (Índice: 2), Similaridad: 0.0711
Palabra similar: '00000000' (Índice: 4), Similaridad: 0.0298

Palabra: 'internet' (Índice: 19254)
Palabra similar: '00' (Índice: 0), Similaridad: 0.0800
Palabra similar: '0000' (Índice: 2), Similaridad: 0.0686
Palabra similar: '00000' (Índice: 3), Similaridad: 0.0594
Palabra similar: '00000000' (Índice: 4), Similaridad: 0.0425

Palabra: 'data' (Índice: 11213)
Palabra similar: '00' (Índice: 0), Similaridad: 0.0711
Palabra similar: '000' (Índice: 1), Similaridad: 0.0686
Palabra similar: '00000000' (Índice: 4), Similaridad: 0.0573
Palabra similar: '00000' (Índice: 3), Similaridad: 0.0555

Palabra: 'science' (Índice: 31051)
Palabra similar: '00000000' (Índice: 4), Similaridad: 0.0987
Palabra similar: '00' (Índice: 0), Similaridad: 0.0940
Palabra similar: '000' (Índice: 1), Similaridad: 